#### Creating a Table for Average Conditions: Pressure, Humdity, Pressure

**Import Functions and Data**

In [0]:
from pyspark.sql.functions import col, when, sum, count, round, avg, percentile, to_date

silverConditions = spark.read.table("gasmonitordata.silver.monitorh2sandconditions")
silverInfo = spark.read.table("gasmonitordata.silver.monitorinformation")

In [0]:
aggConditions = silverConditions.alias('c').join(silverInfo.alias('i'), silverConditions.monitorNum == silverInfo.monitorNum). \
    withColumn('readingDate', to_date(col('c.timeStamp'))). \
    groupBy('c.monitorNum', 'i.groupNum', 'readingDate'). \
    agg(
        round(avg(col('c.tempMH')), 2).alias('avgTempMH'),
        round(percentile(col('c.tempMH'), 0.75), 2).alias('p75_tempMH'),
        round(avg(col('c.humidity')), 2).alias('avgHumidity'),
        round(percentile(col('c.humidity'), 0.75), 2).alias('p75_humidity'),
        round(avg(col('c.pressure')), 2).alias('avgPressure'),
        round(percentile(col('c.pressure'), 0.75), 2).alias('p75_pressure')
    ). \
    orderBy('c.monitorNum', 'i.groupNum', 'readingDate'). \
    select('c.monitorNum', 'i.groupNum', 'readingDate', 'avgTempMH', 'p75_tempMH', 'avgHumidity', 'p75_humidity', 'avgPressure', 'p75_pressure')

In [0]:
aggConditions.write.mode('overwrite').saveAsTable('gasmonitordata.gold.aggConditions')